# Optimization of an agent in Flappy Bird game using Genetic Algorithm

In this project we will develop an agent that plays Flappy Bird using Neural Networks and Genetic Algorithm

# Basic Imports

In [1]:
import pygame
import random
import numpy as np
import torch
import torch.nn as nn

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Constants

In [2]:
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
PIPE_SPEED = 4
GRAVITY = 0.5
BIRD_JUMP = -8
POPULATION_SIZE = 50
MUTATION_RATE = 0.1

# Neural Network

Inputs(5) : y bird position, y top pipe, y lower pipe, horizontal distance, bird velocity

Hidden layer : 16 nodes

Output layer : 1 node

In [ ]:
class BirdBrain(nn.Module):
    def __init__(self):
        super(BirdBrain, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(5, 16),  #5 inputs, hidden layer has 16 nodes
            nn.ReLU(),  #activation
            nn.Linear(16, 1),  #from hidden layer to output layer that determines wether the bird should jump
            nn.Sigmoid(),    #if output is >0.5, JUMP
        )

        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.fc(x)

# Game Logic

**Agent**:

In [ ]:
class Bird:
    def __init__(self):
        self.y = SCREEN_HEIGHT // 2    #y position of the bird
        #normalized (0-1) relative to the screen height


        self.vel = 0
        self.brain = BirdBrain()

        #the longer the bird is alive in the game, the better the fitness
        #GA will use this number to see who is the 'winner'
        #crucial for evolution
        self.fitness = 0 

        self.alive = True 
        self.score = 0

    def jump(self):
        self.vel = BIRD_JUMP

    def update(self, pipe_info):
        '''
            The bird looks at the pipe_info, sends the information to the brain.
            gets an answer (0-1) and decides whether it should jump
        '''

        if not self.alive : return


        '''
            We dont increment fitness for passing through the pipes.
            Because at the beginning all the birds would have 0 points (because none of them know how to pass the pipe). 
            In this way, the one who "survived" longer, even if only by falling, gets a slightly higher grade, which directs the evolution.
        '''
        self.fitness += 0.1 #reward for surviving the frame

        self.vel += GRAVITY # in each frame we add the gravity to the velocity and the velocity to the position
        self.y += self.vel  # this creates a realistic feeling of "falling"

        # DECISION PROCESS - AI
        inputs = torch.tensor(
            [
                self.y / SCREEN_HEIGHT,
                pipe_info['top'] / SCREEN_HEIGHT,
                pipe_info['bottom'] / SCREEN_HEIGHT,
                pipe_info['dist'] / SCREEN_WIDTH,
                self.vel / 15
            ],
            dtype=torch.float32
        )   # normalization (0-1)


        # check if the bird should jump
        if self.brain(inputs).item() > 0.5:
            self.jump()


        # check for collision
        if self.y < 0 or self.y > SCREEN_HEIGHT:
            self.alive = False

